In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel('./item.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_item_number = df['Item_Number'].tolist()
list_category_id = df['Category_Id'].tolist()
list_market_id = df['Market_Id'].tolist()
list_vehicle_page = df['Vehicle_Page'].tolist()

print('总数量：' + str(len(list_item_number)))
list_item_number

In [ ]:
df = pd.DataFrame()

df

In [ ]:
dict_vehicle = {}

dict_vehicle

In [ ]:
list_job = []
for item_number, category_id, market_id, vehicle_page in zip(list_item_number, list_category_id, list_market_id, list_vehicle_page):
    if vehicle_page != '':
        dict_vehicle[item_number] = {}
        dict_vehicle[item_number]['vehicle_page'] = int(vehicle_page)
        for i in range(int(vehicle_page)):
            list_job.append((item_number, category_id, market_id, i))
            
print('总数量：' + str(len(list_job)))
list_job

In [ ]:
import requests
import json

In [ ]:
import sys
sys.path.append('../../../00.module')
import UA
import Proxy

i = 0

item_number, category_id, market_id, vehicle_page = list_job[i]

data = '{"scopedContext":{"catalogDetails":{"itemId":"' + item_number + '","categoryId":"' + category_id + '","marketplaceId":"' + market_id + '"}}}'

ua = UA.get_User_Agent_Requests()
ua['Content-Type'] = 'application/json'

resp = requests.post('https://www.ebay.com/g/api/finders?module_groups=PART_FINDER&referrer=VIEWITEM&offset=' + str(vehicle_page*20) + '&module=COMPATIBILITY_TABLE&_ul=US&_stpos=91710&orig_cvip=true',
                     data=data,
                     headers=ua,
                     proxies=Proxy.get_Proxy_Requests()).text

resp = json.loads(resp)

resp

In [ ]:
list_header = resp['modules']['COMPATIBILITY_TABLE']['paginatedTable']['header']['cells']

list_header

In [ ]:
for k in range(len(list_header)):
    if 'Make' in list_header[k]['textSpans'][0]['text']:
        col_make = k
    elif 'Model' in list_header[k]['textSpans'][0]['text']:
        col_model = k
    elif 'Year' in list_header[k]['textSpans'][0]['text']:
        col_year = k
        
col_make, col_model, col_year

In [ ]:
list_row = resp['modules']['COMPATIBILITY_TABLE']['paginatedTable']['rows']

list_row

In [ ]:
list_vehicle = []
for row in list_row:
    make = row['cells'][col_make]['textSpans'][0]['text'].strip()
    model = row['cells'][col_model]['textSpans'][0]['text'].strip()
    year = int(row['cells'][col_year]['textSpans'][0]['text'].strip())
    
    list_vehicle.append({'make': make, 'model': model, 'year': year})
            
dict_vehicle[item_number][str(vehicle_page)] = list_vehicle

list_vehicle

In [ ]:
if dict_vehicle[item_number]['vehicle_page']+1 == len(dict_vehicle[item_number]):
    print('fuck')

In [ ]:
new_list_vehicle = []
new_dict_vehicle = {}
for i in range(dict_vehicle[item_number]['vehicle_page']):
    list_vehicle += dict_vehicle[item_number][str(i)]
    
for vehicle in list_vehicle:
    if vehicle['make'] not in new_dict_vehicle:
        new_dict_vehicle[vehicle['make']] = {vehicle['model']: [vehicle['year']]}
    else:
        if vehicle['model'] not in new_dict_vehicle[vehicle['make']]:
            new_dict_vehicle[vehicle['make']][vehicle['model']] = [vehicle['year']]
        else:
            new_dict_vehicle[vehicle['make']][vehicle['model']].append(vehicle['year'])
            
new_dict_vehicle

In [ ]:
new_list_vehicle = []
for make in new_dict_vehicle:
    for model in new_dict_vehicle[make]:
        year_max = max(new_dict_vehicle[make][model])
        year_min = min(new_dict_vehicle[make][model])
        if year_max == year_min:
            new_list_vehicle.append(make + ' ' + model + ' ' + str(year_min))
        else:
            new_list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))
            
vehicle = '\n'.join(new_list_vehicle)

print(vehicle)

In [ ]:
df_temp = pd.DataFrame([{'status': 'ok',
                         'Item_Number': item_number,
                         'Vehicle': vehicle}])

df = pd.concat([df, df_temp], ignore_index=True).fillna('')

df